In [1]:
import pandas as pd
import pyranges as pr

In [2]:
# fname = '/Users/fairliereese/mortazavi_lab/ref/gencode.v39/gencode.v39.annotation.gtf'
# v39 = pr.read_gtf(fname, duplicate_attr=True)

In [3]:
# v39.head()

In [19]:
fname = '/Users/fairliereese/mortazavi_lab/data/rnawg/lr_bulk/talon/human_known_nic_nnc_talon.gtf'
print('this block is running')
# fname = '/Users/fairliereese/mortazavi_lab/data/rnawg/lr_bulk/talon/test.gtf'
talon = pr.read_gtf(fname, duplicate_attr=True)

this block is running


In [20]:
talon = talon.df

In [21]:
talon.head()

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_name,...,tag,transcript_type,transcript_support_level,havana_transcript,NNC_transcript,protein_id,ccdsid,intergenic_novel,antisense_gene,gene_antisense_to_IDs
0,SIRV1,TALON,gene,10647,11606,.,+,.,ENCODEHG000058846,ENCODEHG000058846,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TRUE,NaN,NaN
1,SIRV1,TALON,transcript,10647,11606,.,+,.,ENCODEHG000058846,ENCODEHG000058846,...,NaN,NaN,NaN,NaN,TRUE,NaN,NaN,NaN,NaN,NaN
2,SIRV1,TALON,exon,10647,10791,.,+,.,ENCODEHG000058846,ENCODEHG000058846,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SIRV1,TALON,exon,10882,11057,.,+,.,ENCODEHG000058846,ENCODEHG000058846,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SIRV1,TALON,exon,11434,11606,.,+,.,ENCODEHG000058846,ENCODEHG000058846,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
talon.loc[talon.transcript_id == 'ENCODEHT000206785']

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_name,...,tag,transcript_type,transcript_support_level,havana_transcript,NNC_transcript,protein_id,ccdsid,intergenic_novel,antisense_gene,gene_antisense_to_IDs
30,SIRV2,TALON,transcript,995,5906,.,-,.,ENCODEHG000058784,ENCODEHG000058784,...,NaN,NaN,NaN,NaN,TRUE,NaN,NaN,NaN,NaN,NaN
31,SIRV2,TALON,exon,5788,5906,.,-,.,ENCODEHG000058784,ENCODEHG000058784,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,SIRV2,TALON,exon,4687,4800,.,-,.,ENCODEHG000058784,ENCODEHG000058784,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,SIRV2,TALON,exon,4338,4479,.,-,.,ENCODEHG000058784,ENCODEHG000058784,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,SIRV2,TALON,exon,3966,4094,.,-,.,ENCODEHG000058784,ENCODEHG000058784,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,SIRV2,TALON,exon,3665,3825,.,-,.,ENCODEHG000058784,ENCODEHG000058784,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,SIRV2,TALON,exon,3105,3325,.,-,.,ENCODEHG000058784,ENCODEHG000058784,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,SIRV2,TALON,exon,2881,3010,.,-,.,ENCODEHG000058784,ENCODEHG000058784,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,SIRV2,TALON,exon,2674,2802,.,-,.,ENCODEHG000058784,ENCODEHG000058784,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,SIRV2,TALON,exon,1973,2064,.,-,.,ENCODEHG000058784,ENCODEHG000058784,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
talon.loc[(talon.gene_id == 'ENCODEHG000058784')&(talon.Start == 5788)].transcript_id

31    ENCODEHT000206785
Name: transcript_id, dtype: object

In [30]:
def get_ic(gtf_pr):
    """
    Get a hyphen-separated representation of each transcript's intron chain
    from a PyRanges GTF

    Parameters:
        gtf_pr (pyranges PyRanges): GTF PyRanges object

    Returns:
        df (pandas DataFrame): DataFrame detailing intron chain, gene, strand,
            chromosome, and transcript that intron chain was seen in
    """
    df = gtf_pr.df.copy(deep=True)

    # restrict to exon entries
    df = df.loc[df.Feature == 'exon']
    cols = ['Chromosome', 'Strand', 'Start', 'End', 'transcript_id', 'gene_id']
    df = df[cols]

    # melt to isolate individual coordinates
    df = pd.melt(df, id_vars=['Chromosome', 'Strand', 'transcript_id', 'gene_id'],
                value_vars=['Start', 'End'],
                value_name='Coord')
    df.drop('variable', axis=1, inplace=True)

    # sort to order coordinates correctly
    df.Coord = df.Coord.astype(int)
    fwd = df.loc[df.Strand == '+'].copy(deep=True)
    rev = df.loc[df.Strand == '-'].copy(deep=True)

    fwd.sort_values(by=['Chromosome', 'transcript_id', 'Coord'],
                    ascending=[True, True, True], inplace=True)
    rev.sort_values(by=['Chromosome', 'transcript_id', 'Coord'],
                    ascending=[True, True, False], inplace=True)
    df = pd.concat([fwd, rev])

    # create intron chain strings
    df.Coord = df.Coord.astype(str)
    df = df.groupby(['Chromosome', 'Strand',
                     'transcript_id', 'gene_id'], observed=True)['Coord'].apply('-'.join).reset_index()

    # remove tss and tes from intron chain
    df['temp'] = df.Coord.str.split('-', n=1, expand=True)[1]
    df['ic'] = df.temp.str.rsplit('-', n=1, expand=True)[0]

    return df

In [25]:
df.loc[df.transcript_id == 'ENCODEHT000206785']

,Chromosome,Strand,transcript_id,gene_id,Coord,temp,ic
127212,SIRV2,-,ENCODEHT000206785,ENCODEHG000058784,5906-5788-4800-4687-4479-4338-4094-3966-3825-3...,5788-4800-4687-4479-4338-4094-3966-3825-3665-3...,5788-4800-4687-4479-4338-4094-3966-3825-3665-3...


In [29]:
df.loc[df.Coord.isnull()]

,Chromosome,Strand,transcript_id,gene_id,Coord,temp,ic


In [34]:
df = talon.copy(deep=True)
df = get_ic(df)

In [35]:
df.head()

,Chromosome,Strand,transcript_id,gene_id,Coord,temp,ic
0,SIRV1,+,ENCODEHT000206942,ENCODEHG000058846,10647-10791-10882-11057-11434-11606,10791-10882-11057-11434-11606,10791-10882-11057-11434
1,SIRV1,-,ENCODEHT000206867,ENCODEHG000058837,10790-10647-10366-10282-7814-7552-6813-6560-64...,10647-10366-10282-7814-7552-6813-6560-6473-633...,10647-10366-10282-7814-7552-6813-6560-6473-633...
2,SIRV1,-,ENCODEHT000206868,ENCODEHG000058837,10790-10444-10366-10282-7814-7552-6813-6560-64...,10444-10366-10282-7814-7552-6813-6560-6473-633...,10444-10366-10282-7814-7552-6813-6560-6473-633...
3,SIRV1,-,ENCODEHT000206870,ENCODEHG000058837,10790-10553-7808-7552-1484-999,10553-7808-7552-1484-999,10553-7808-7552-1484
4,SIRV1,-,ENCODEHT000206886,ENCODEHG000058837,10366-10282-7814-7552-6813-6337-1484-999,10282-7814-7552-6813-6337-1484-999,10282-7814-7552-6813-6337-1484


In [10]:
talon.head()

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_name,...,tag,transcript_type,transcript_support_level,havana_transcript,NNC_transcript,protein_id,ccdsid,intergenic_novel,antisense_gene,gene_antisense_to_IDs
0,SIRV1,TALON,gene,10647,11606,.,+,.,ENCODEHG000058846,ENCODEHG000058846,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TRUE,NaN,NaN
1,SIRV1,TALON,transcript,10647,11606,.,+,.,ENCODEHG000058846,ENCODEHG000058846,...,NaN,NaN,NaN,NaN,TRUE,NaN,NaN,NaN,NaN,NaN
2,SIRV1,TALON,exon,10647,10791,.,+,.,ENCODEHG000058846,ENCODEHG000058846,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SIRV1,TALON,exon,10882,11057,.,+,.,ENCODEHG000058846,ENCODEHG000058846,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SIRV1,TALON,exon,11434,11606,.,+,.,ENCODEHG000058846,ENCODEHG000058846,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,SIRV1,TALON,gene,999,10366,.,-,.,ENCODEHG000058837,ENCODEHG000058837,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TRUE,NaN,NaN
6,SIRV1,TALON,transcript,999,10790,.,-,.,ENCODEHG000058837,ENCODEHG000058837,...,NaN,NaN,NaN,NaN,TRUE,NaN,NaN,NaN,NaN,NaN
7,SIRV1,TALON,exon,10647,10790,.,-,.,ENCODEHG000058837,ENCODEHG000058837,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
talon = talon.df
talon.loc[talon.transcript_id == 'ENCODEHT000206785']

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_name,...,tag,transcript_type,transcript_support_level,havana_transcript,NNC_transcript,protein_id,ccdsid,intergenic_novel,antisense_gene,gene_antisense_to_IDs
30,SIRV2,TALON,transcript,995,5906,.,-,.,ENCODEHG000058784,ENCODEHG000058784,...,NaN,NaN,NaN,NaN,TRUE,NaN,NaN,NaN,NaN,NaN
31,SIRV2,TALON,exon,5788,5906,.,-,.,ENCODEHG000058784,ENCODEHG000058784,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,SIRV2,TALON,exon,4687,4800,.,-,.,ENCODEHG000058784,ENCODEHG000058784,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,SIRV2,TALON,exon,4338,4479,.,-,.,ENCODEHG000058784,ENCODEHG000058784,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,SIRV2,TALON,exon,3966,4094,.,-,.,ENCODEHG000058784,ENCODEHG000058784,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,SIRV2,TALON,exon,3665,3825,.,-,.,ENCODEHG000058784,ENCODEHG000058784,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,SIRV2,TALON,exon,3105,3325,.,-,.,ENCODEHG000058784,ENCODEHG000058784,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,SIRV2,TALON,exon,2881,3010,.,-,.,ENCODEHG000058784,ENCODEHG000058784,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,SIRV2,TALON,exon,2674,2802,.,-,.,ENCODEHG000058784,ENCODEHG000058784,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,SIRV2,TALON,exon,1973,2064,.,-,.,ENCODEHG000058784,ENCODEHG000058784,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
